# Introduzione

# Importazione pacchetti

Importare i vari pacchetti necessari  per eseguire le azioni elencate nell'introduzione:
   - bs4 (beautiful soup, manipolare gli html)
   - tqdm (per le barre di progresso) 
   - pandas (manipolazione/esportazione dataframes)


In [ ]:
from bs4 import BeautifulSoup # Importazione pacchetti per gestire HTML
from tqdm import tqdm as tqdm # Importazione pacchetti per barra di progresso
import pandas as pd  # Importazione pacchetti per lavorare con i dataframe
import numpy as np # Usato per nan

#pd.set_option('display.max_rows', None)

In [ ]:
# Directory principale dello scraper, ove vi sono gli script e gli html devono essere posizionati
path_main_directory = '...'

# Directory del progetto
path_progetto = '...'

In [ ]:
# Inizializzare il dataframe nel quale verranno inserite le informazioni riguardo gli hotel e  B&B di agoda.
booking_dataframe = pd.DataFrame(columns=[
   "titolo",
   "recensione_voto_numerico",
   "recensione_voto_parola",
   "numero_recensioni",
   "date",
   "permanenza",
   "prezzo",
   "indirizzo",
   "valutazione"
])


# Ciclo for per il numero di pagine dalle quali estrarre i dati.
for i in range(1,7):

   # Importare l'html del file in questione
   with open(f"{path_main_directory}/html/html_final_pagina{i}.txt","r") as f:
      html = f.read()

   # Prendere l'html della pagina 
   html_bs4 = BeautifulSoup(html,'html.parser')
   proprietà = html_bs4.find_all("div",{"data-element-name":"PropertyCardBaseJacket"})


   for post in tqdm(proprietà):
      
      # DIV info proprietà
      header = post.find("header",{"data-element-name":"property-info-header"})
      
      # Titolo
      try: # Se non ha il titolo non è valida, passare alla successiva
         titolo = header.find("h3",{"data-selenium":"hotel-name"}).text
      except:
         continue 

      #  rating-container
      rating = header.find("div",{"data-testid":"rating-container"})
      if rating != None:
         rating = rating.find_all("svg",{"opacity":"1"})
         stelle = len(rating)
      else:
         stelle = np.nan

      
      indirizzo = header.find("div",{"data-selenium":"area-city"}).text
      indirizzo = indirizzo.replace("- Guarda sulla mappa","")
      
      # Prezzi e recensioni
      informazioni = post.find("div",{"data-element-name":"property-card-pricing-info"})
      
      ## Recensioni
      informazioni_recensioni = informazioni.find("div",{"class":"Box-sc-kv6pi1-0 hRUYUu PropertyCard__PricingHeadliner"})
   
      ### Questa div è molto caotica, bisogna cercare le tag p che contengono le recensioni
      
      for p in informazioni_recensioni.find_all("p"):
         
         spans = p.find_all("span")
         if  len(spans) > 0: # se ci sono più span nella tag p allora si tratta dei voti numerici o parola
            for span in spans:
               if "," in span.text: # se c'è una virgola è voto numerico
                  recensione_voto_numerico = span.text
               else: # se non c'è la virgola è voto parola
                  recensione_voto_parola = span.text
         else: # se non ci sono span è il numero di recensioni
            numero_recensioni = p.text

      # Prezzo
      prezzo = informazioni.find("div",{"data-element-name":"property-card-price"})
      prezzo = prezzo.find("div",{"data-element-name":"final-price"}).text


      # Crea il dizionario con i dati della singola proprietà
      informazioni_singola_proprietà = {  "titolo": titolo,
                                          "recensione_voto_numerico": recensione_voto_numerico,
                                          "recensione_voto_parola": recensione_voto_parola,
                                          "numero_recensioni": numero_recensioni,
                                          "date": "1 agosto - 2 agosto",
                                          "permanenza": "1 notte, 2 adulti",
                                          "prezzo": prezzo,
                                          "indirizzo": indirizzo,
                                          "valutazione": stelle,
                                       }
      
      # Aggiunge direttamente come nuova riga
      booking_dataframe.loc[len(booking_dataframe)] = informazioni_singola_proprietà


100%|██████████| 90/90 [00:00<00:00, 973.33it/s]


In [ ]:
# Esportare il dataframe
booking_dataframe.to_csv(f"{path_progetto}/Data Lake/agodaRoma.csv")

In [ ]:
#pd.set_option('display.max_rows', None)
#booking_dataframe.sort_values(by="titolo")